<h1 align="center">An Introduction to Machine Learning - 25737</h1>
<h4 align="center">Dr. Yassaee</h4>
<h4 align="center">Sharif University of Technology, Autumn 2024</h4>

**Student Name**:

**Student ID**:

# Non-Linear Classification with SVM

## 1. DATA SET AND VISUALIZATION FUNCTION
We will use the non-linear toy data called the Moon dataset. You may use the code snippet below to generate the train/test set. Feel free to change the number of samples, and noise level. Additionally, a function plot_svm() is provided to help you visualize the decision boundary, margin, and support vectors on the dataset in 2D feature space.

The provided function plot_svm() works out-the-box, and is the best way to visualize and evaluate the performance of your model. It assumes the classifier has an instance variable "self.support_vectors_", which is a numpy array of the support vectors found in training. DO NOT modify this function. Once your implementation in Task 3 is complete, the plots generated for your model should look similar to the plots generated for the standard library models in Task 2.

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from scipy.stats import randint, uniform
import numpy as np
import cvxopt # The optimization package for Quadratic Programming
import cvxopt.solvers
from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
# Generate a non-linear Moon dataset
X, y = make_moons(n_samples=500, noise=0.15, random_state=49)

# Convert labels from {0,1} to {-1, +1}
# TODO: Convert the binary labels y from {0,1} to {-1, +1}

# Split dataset into training and test sets
# TODO: Use train_test_split to split the data into train and test sets (80% training, 20% test) with random_state of 42.


def plot_svm(clf, X, y, axes=[-2, 3, -2, 2]):
    """
    Generate a simple plot of SVM including the decision boundary, margin, and its training data

    Parameters
    ----------
    clf: your classifier handle
    X: feature matrix shape(m_samples, n_features)
    y: label vector shape(m_samples, )
    axes: (optional) the axes of the plot in format [xmin, xmax, ymin, ymax]
    """
    # TODO: Create a mesh grid for x0 and x1 axes based on the provided axes (100 x 100 resolution).


    # TODO: Combine the grid into a feature matrix (2-D points) and make predictions using clf for the mesh points.



    # TODO: Set up the plot dimensions (e.g., 16x9).

    # TODO: Plot data points for each class


- - -
## 2. TRAIN SVM FOR CLASSIFICATION TASK (20 pts)

Use the standard libarary SVM classifier (SVC) on the training data, and then test the classifier on the test data. You will need to call SVM with 3 kernels: (1) Linear, (2) Polynomial and (3) Gaussian RBF.

You should tune each model using a grid search or similar hyperparameter selection process, and report the best hyperparameters found. You will use these same hyperparameter settings later when testing and comparing to your implementation in Task 4. Once you've selected the best hyperparameters for each kernel, you will need to report the following:

* Confusion matrix, Recall, and Precision. If applicable, discuss any tuning process on C and/or gamma to get to a reasonable result.
* Use the provided plot_svm() to visualize the SVM in 2D. This might give you some insight on how SVM determines the margin and support vector on the Moon dataset.


In [ ]:
def tuneAndTest(model, params):
  rnd_search = RandomizedSearchCV(model, param_distributions =params, n_iter = 50, cv = 5, random_state=40)
  rnd_search.fit(X_train, y_train)
  print("best hyper-parameter value: ", rnd_search.best_params_)
  bestmodel = rnd_search.best_estimator_

  #predict
  prediction = bestmodel.predict(X_test)
  # print("model:", str(bestmodel))
  print("confusion matrix:", confusion_matrix(y_test, prediction))
  print("recall:",recall_score(y_test, prediction))
  print("precision:",precision_score(y_test, prediction))
  plot_svm(bestmodel, X_train, y_train)
  print("\n")




In [ ]:
def tuneAndTest(model, params):
    """
    Perform hyperparameter tuning using RandomizedSearchCV and evaluate the model on the test set.

    Parameters:
    ----------
    model : the SVM model instance (SVC)
    params : the dictionary of hyperparameters to tune for this model

    """
    # TODO: Create RandomizedSearchCV instance with 50 iterations, 5-fold cross-validation, and a random state of 40.

    # TODO: Fit the RandomizedSearchCV instance to X_train and y_train.

    # TODO: Predict the labels for the test set (X_test).

    # TODO: Print the confusion matrix for y_test vs. predicted values, along with recall and precision scores.

    # Visualize decision boundary with plot_svm
    # TODO: Plot the SVM model using plot_svm function to visualize decision boundaries and support vectors.


# TODO: Define parameter grid for each kernel type (linear, polynomial, and RBF)
# Linear kernel: parameters for tuning C

# Polynomial kernel: parameters for tuning C, degree, and coef0

# RBF kernel: parameters for tuning C and gamma

# Create pipelines for each kernel type
# TODO: Initialize SVC model with each kernel type in a separate pipeline with StandardScaler for scaling.

# TODO: Call tuneAndTest function for each kernel type, passing in the respective SVM model and parameter grid.


In [ ]:
tuneAndTest(SVC(kernel="linear"),{"C": uniform(0,20)})

In [ ]:
tuneAndTest(SVC(kernel="poly"),{"coef0":randint(low=0, high=5),"C": uniform(0,20),"degree": randint(low=2, high=5), "gamma":uniform(0, 1)})

In [ ]:
tuneAndTest(SVC(kernel="rbf"),{"C": uniform(0,20), "gamma":uniform(0, 1)})

## 3. IMPLEMENT YOUR OWN NON-LINEAR SVM (60 pts)
Now that you see how the standard library SVM perform on the dataset, you will attempt to implement your own version of SVM. To help you, a template of SVM has been created including the quadratic optimization. Essensially, you will get the optimized value of $\alpha$ for free.

The provided code is extensively documented in comments, so that you may write code compatible with it. DO NOT edit the provided code.

Many of these tasks are made much easier by a working knowledge of numpy. If you have something you want to do, but are unsure how to do it in Python, consult the numpy documentation here: https://numpy.org/doc/1.17/reference/index.html. If you haven't built a Python class before, it may be worth scanning this tutorial: https://www.datacamp.com/community/tutorials/python-oop-tutorial.

In [ ]:
# Use the information from the lecture slides to formulate the SVM Kernels.
# To help you get started, the Linear Kernel (simply just a dot product) has been provided to you.
# These kernel functions will be called in the SVM class
# Linear Kernel
def linear_kernel(u, v):
    return np.dot(u, v)

# Polynomial Kernel (of degree up to and including p)
def polynomial_kernel(u, v, p=3):
    return np.dot(u,v)**p

# Gaussian RBF Kernel
def rbf_kernel(u, v, gamma=0.1):
    # Note that gamma is provided, not sigma; see the slides for the relationship between gamma and sigma
    return np.exp(-gamma*np.linalg.norm(u-v)**2)

In [ ]:
class MySVM(BaseEstimator, ClassifierMixin):
    """The Implementation of the SVM class"""

    def __init__(self, kernel=linear_kernel, C=None):
        self.kernel = kernel  # Kernel function, can be called within the class
        self.C = C
        if self.C is not None:
            self.C = float(self.C)

    def fit(self, X, y=None):
        """
        Train SVM based on the training set
        Parameters
        ----------
        X: feature matrix shape(m_samples, n_features)
        y: label vector shape(m_samples, )
        """
        self.X = X
        self.y = y
        m_samples, n_features = X.shape

        # Compute the kernel matrix
        K = np.zeros((m_samples, m_samples))
        for i in range(m_samples):
            for j in range(m_samples):
                K[i, j] = self.kernel(X[i], X[j])

        # Define Quadratic Programming parameters
        P = cvxopt.matrix(np.outer(y, y) * K)
        q = cvxopt.matrix(np.ones(m_samples) * -1)
        A = cvxopt.matrix(y, (1, m_samples))
        b = cvxopt.matrix(0.0)

        # TODO: If self.C is None, set G and h for a hard margin SVM. Else, define them for soft margin SVM.

        # Solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        a = np.ravel(solution['x'])

        # TODO: Identify support vectors and store them in self.support_vectors_

        if self.kernel == linear_kernel:
            ay = (a * y)[:, np.newaxis]
            self.w = np.dot(ay.T, X).T
        else:
            # No weight vector needed for non-linear case.
            self.w = None

        # TODO: Compute the intercept b based on support vectors.

    def decision_function(self, X):
        """The decision function is essentially w^T . x + b"""
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            # TODO: Implement the kernel trick to predict label for non-linear SVM

    def predict(self, X):
        """Predicts -1,+1 based on the sign of the decision function"""
        return np.sign(self.decision_function(X))


---
## 4. COMPARE YOUR IMPLEMENTATION TO THE STANDARD LIBRARY (20 pts)
Now that you have implemented your own SVM class, let's use it! Create 3 instances of your SVM class, each with a difference kernel (Linear, Polynomial, and RBF kernel), then train and test its performance in the Moon dataset as above. Use the same hyperparameters found to be best for the standard library implementations, and report the same metrics (confusion matrix, recall, and precision). You can use the plot_svm() function to visualize your SVM with decision boundary, margin, and support vectors on the dataset, and should plot each of the three instances of your class, to compare these plots with the standard library models' plots.

Based on the number above, compare your SVM implementation with the standard library version. How did your SVM perform in comparison? Is there any major differences between the algorithms? If your performance is significantly worse, is there a different set of hyperparameters which better fits your model? Finally, reflect on your experience implementing a learning algorithm for this assignment.

In [ ]:
import matplotlib.pyplot as plt

def plot_svm(clf, X, y, axes=[-2, 3, -2, 2]):
    """
    Visualizes the SVM decision boundary, margins, and support vectors.
    """
    # TODO: Set up a mesh grid over the specified axes to visualize decision boundaries

    # TODO: Flatten the mesh grid to feed into clf.predict and clf.decision_function for visualization

    # Plotting
    plt.figure(figsize=(16, 9))
    plt.plot(X[:, 0][y == -1], X[:, 1][y == -1], "bo", label="Class -1")
    plt.plot(X[:, 0][y == 1], X[:, 1][y == 1], "go", label="Class +1")

    # TODO: Use clf.support_vectors_ to plot support vectors

    # TODO: Use contour plotting to show the decision boundary and margins


def printResult(model):
    """Displays confusion matrix, recall, precision, and plots SVM boundaries."""
    # TODO: Use model.predict() to generate predictions for the test set

    # TODO: Print out evaluation metrics (confusion matrix, recall, precision) for the model

    # TODO: Use plot_svm to visualize the decision boundary, support vectors, and margins


def comp(mysvm, lib):
    """Compares the custom SVM with the standard library SVM implementation."""
    # TODO: Fit both the custom SVM and library SVC model on the training data

    print("Custom SVM Implementation:")
    print(mysvm)
    # TODO: Call printResult to display the results of the custom SVM

    print("Standard Library Implementation:")
    # TODO: Call printResult to display the results of the library SVC model


In [ ]:
custom_linear_svm = MySVM(kernel=linear_kernel, C=8.1537)

In [ ]:
comp(custom_linear_svm, SVC(kernel="linear", C=8.1537))

In [ ]:
custom_poly_svm = MySVM(kernel=lambda u, v: polynomial_kernel(u, v, p=3), C=0.5)

In [ ]:
comp(custom_poly_svm, SVC(kernel="poly", C=0.5, degree=3))

In [ ]:
custom_rbf_svm = MySVM(kernel=lambda u, v: rbf_kernel(u, v, gamma=0.1), C=1.5)

In [ ]:
comp(custom_rbf_svm, SVC(kernel="rbf", C=1.5, gamma=0.1))